## Setup

In [1]:
import dotenv, os, openai, csv

EXAMPLES_DIR = 'examples'
RESULTS_FILENAME = 'results.csv'

dotenv.load_dotenv()
gpt = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def prompt_gpt(prompt: str) -> str:
    response = gpt.chat.completions.create(
        model='gpt-3.5',
        messages=[
            {
                'role': 'user',
                'content': prompt,
            }
        ]
    )
    output = response.choices[0].message.content.strip()
    return output

def format_examples(examples: list[dict[str, str]]) -> str:
    formatted_examples = ""
    
    for example in examples:
        formatted_examples += f"Q: {example['q']}\nA: {example['a']}\n"
    
    return formatted_examples

def run_trial(case: str, task_class: str, examples: list[dict[str, str]], task: str, results:  list[list[str]]) -> None:
    formatted_examples = format_examples(examples)
    prompt = f"{formatted_examples}Q: {task}"  # formatted_examples ends in new line character
    result = prompt_gpt(prompt)
    results.append([task_class, case, result])

## Load examples and define tasks

Examples are stored in CSV files, formatted as follows:
```
q_1,a_1
q2,a_2
...
q_n,a_n
```

In [2]:
tasks_by_task_class = {  # TODO add tasks (questions that will follow examples)
    # "Math": "",
    # "CSQA": "",
    # "StrategyQA": "",
    # "Dates": "",
    # "Sports": "",
    # "SayCan": ""
}

examples_by_task_class = {}  # TODO add examples into CSVs (ideally 12 or 16 total)

for task_class in tasks_by_task_class.keys():
    path = os.path.join(EXAMPLES_DIR, f'{task_class}.csv')
    
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        examples_by_task_class[task_class] = [{'q': row[0], 'a': row[1]} for row in reader]

FileNotFoundError: [Errno 2] No such file or directory: 'examples/Math.csv'

## Run trials

In [ ]:
results = [["Task Class", "Case", "Response"]]

for task_class, examples in examples_by_task_class.items():
    task = tasks_by_task_class[task_class]
    half_n = len(examples) // 2
    # case 1: use n / 2 examples
        # case 1a: use the first n / 2 examples
    case1a_examples = examples[:half_n]
    run_trial("1a", task_class, case1a_examples, task, results)
        # case 1b: use the other n / 2 examples
    case1b_examples = examples[half_n:]
    run_trial("1a", task_class, case1b_examples, task, results)
    # case 2: use all n examples
    case2_examples = examples
    run_trial("2", task_class, case2_examples, task, results)
    # case 3: use n / 2 examples, duplicated (for n total examples)
        # case 3a: use the first n / 2 examples
    case3a_examples = examples[:half_n] + examples[:half_n]
    run_trial("3a", task_class, case3a_examples, task, results)
        # case 3b: use the other n / 2 examples
    case3b_examples = examples[half_n:] + examples[half_n:]
    run_trial("3b", task_class, case3b_examples, task, results)
    # TODO other cases

with open(RESULTS_FILENAME, 'w', newline='') as f:
    writer = csv.writer(f)
    
    for row in results:
        writer.writerow(row)